In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def xlsread(filename, sheetname, _usecols, _userows):
    df = pd.read_excel(filename, sheet_name=sheetname, usecols=_usecols)
    df = df.loc[_userows[0]:_userows[1]]
    return df.values

# 由于论文展示数据未完全，只用到四大蔬菜品类相关数据，且其实际为某一单品的数据而非整个品类
S = xlsread('未来一周六大类日销预测.xlsx', 'Sheet1', 'B:E', [0,6]).transpose()
L = xlsread('六大蔬菜品类平均损耗率.xlsx', 'Sheet1', 'A:D', [0,0])[0]
B = xlsread('六大蔬菜品类每日进货价.xlsx', 'Sheet1', 'A:D', [0,0])[0]
print("S:", S)
print("L:", L)
print("B:", B)

def Gain(x):
    w = 0
    for i in range(4):
        for j in range(7):
            w += S[i, j] * (x[i, 0] - B[i]) - L[i] * x[i, 1] * B[i]
    return w

narvs = 2 ## %1: 售价 %2：进货量
T0 = 100
T = T0
maxgen = 200
Lk = 100
alfa = 0.95

x_lb = np.zeros((narvs, 4))
x_ub = np.zeros((narvs, 4))
for i in range(4):
    x_lb[0, i] = (B[i] * 1.1)
    x_ub[0, i] = (B[i] * 1.5)
    # 缺乏合适的数据，先用定值
    x_lb[1, i] = S[i, 0]
x_ub[1] = [2.5, 2.5, 2.5, 4.5]

print("上界", x_ub)
print("下界", x_lb)

x0 = np.zeros((4, narvs))
for i in range(narvs):
    for j in range(4):
        x0[j, i] = x_lb[i, j] + (x_ub[i, j] - x_lb[i, j]) * np.random.rand(1)
y0 = Gain(x0)
max_y = y0
MAXY = np.zeros((maxgen, 1))

for iter in range(maxgen):  # 外循环
    for i in range(Lk):     # 内循环
        y = np.random.randn(1, narvs)
        z = y / np.sqrt(np.sum(y**2))
        x_new = x0 + z * T
        for i in range(4):
            for j in range(narvs):
                if x_new[i, j] < x_lb[j, i]:
                    r = np.random.rand(1)
                    x_new[i, j] = r * x_lb[j, i] + (1 - r) * x0[i, j]
                elif x_new[i, j] > x_ub[j, i]:
                    r = np.random.rand(1)
                    x_new[i, j] = r * x_ub[j, i] + (1 - r) * x0[i, j]
        x1 = x_new
        y1 = Gain(x1)
        if y1 > y0:
            x0 = x1
            y0 = y1
        else:
            p = np.exp(-(y0 - y1) / T)
            if np.random.rand(1) < p:
                x0 = x1
                y0 = y1
        if y0 > max_y:
            max_y = y0
            best_x = x0
    MAXY[iter] = max_y
    T = alfa * T

print('最佳的位置是：', best_x)
print('此时最优值是：', max_y)

S: [[1.15 1.15 1.15 1.15 1.15 1.15 1.15]
 [1.4  1.4  1.4  1.4  1.4  1.4  1.4 ]
 [0.25 0.25 0.25 0.25 0.25 0.25 0.25]
 [0.5  0.5  0.5  0.5  0.5  0.5  0.5 ]]
L: [0.1525 0.0943 0.1607 0.0761]
B: [3.29 4.59 8.09 1.4 ]
上界 [[ 4.935  6.885 12.135  2.1  ]
 [ 2.5    2.5    2.5    4.5  ]]
下界 [[3.619 5.049 8.899 1.54 ]
 [1.15  1.4   0.25  0.5  ]]


/tmp/ipykernel_23253/4017950372.py:47: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x0[j, i] = x_lb[i, j] + (x_ub[i, j] - x_lb[i, j]) * np.random.rand(1)
/tmp/ipykernel_23253/4017950372.py:61: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x_new[i, j] = r * x_lb[j, i] + (1 - r) * x0[i, j]
/tmp/ipykernel_23253/4017950372.py:64: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x_new[i, j] = r * x_ub[j, i] + (1 - r) * x0[i, j]


最佳的位置是： [[ 4.935  1.15 ]
 [ 6.885  1.4  ]
 [12.135  0.25 ]
 [ 2.1    0.5  ]]
此时最优值是： 34.33331089999999
